# **Analizador léxico-sintáctico**

Librerías importantes , si en llegado caso , descomentar el install de tkinter

In [ ]:
!pip install ttkbootstrap
!pip install tk
# !pip install tkinter

## **1.Analizador léxico**

In [ ]:
import re

class AnalizadorLexico:
    def __init__(self):
        self.pseudocode = []

    def search_for(self, txt: str) -> list:
        for_pattern = re.compile(r"(Para.*Hasta.+FinPara)")
        return for_pattern.findall(txt)

    def search_def(self, txt: str) -> list:
        def_pattern = re.compile(r".+[a-zA-Z0-9][^$!%&=?#]+Entero|Cadena|lógico|Real+[;]")
        return def_pattern.findall(txt)

    def search_if(self, txt: str) -> list:
        if_pattern = re.compile(r"(Si.*Entonces.*FinSi)")
        return if_pattern.findall(txt)

    def search_while(self, txt: str) -> list:
        while_pattern = re.compile(r"(Mientras.*Hacer.*FinMientras)")
        return while_pattern.findall(txt)

    def search_do_while(self, txt: str) -> list:
        do_while_pattern = re.compile(r"(Repetir.*Hasta Que.*FinRepetir)")
        return do_while_pattern.findall(txt)

    def search_wr(self, txt: str) -> list:
        wr_pattern = re.compile(r"(Leer|Escribir.+[a-zA-Z-0-9-,]+[a-zA-Z-0-9][;])")
        return wr_pattern.findall(txt)

    def analisis(self, txt: str):
        self.pseudocode = []
        self.pseudocode.append(self.search_for(txt))
        self.pseudocode.append(self.search_def(txt))
        self.pseudocode.append(self.search_if(txt))
        self.pseudocode.append(self.search_while(txt))
        self.pseudocode.append(self.search_do_while(txt))
        self.pseudocode.append(self.search_wr(txt))
        return self.pseudocode


## **2.Analizador Sinctactico**

In [ ]:
import re

class error(Exception):
    def __init__(self,mensaje:str):
        super().__init__(mensaje)
        self.mensajes=mensaje
    def __str__(self):
        return f'{self.mensajes}'
        

class AnalizadorSintactico:
    def __init__(self,codigo:list):
        self.error_actual='empty'
        self.indice_actual = 0
        self.codigo=codigo[0] #lista con la lista a analizar
        self.indice=0

    def reiniciar(self):
        self.error_actual='empty'
        self.indice=0
    
        
    def obtener_siguiente_cadena(self,estructura:str):
        cadenas=estructura.split(" ")
        if cadenas and self.indice < len(cadenas):
            estructura_actual = cadenas[self.indice]
            self.indice += 1
            return estructura_actual
        else:
            self.indice = 0
            return None
      
    ##Funciones para validar las estructuras  
    
    def alfanumerico(self,cadena:str):
        regex=re.compile('[a-zA-Z0-9_]+;')
        if re.fullmatch(regex,cadena):
            return True
        else:
            return False
    
    def valor_numerico(self,cadena:str):
        regex=re.compile('[0-9]+')
        if re.fullmatch(regex,cadena):
            return True
        else:
            return False
    
    def paso_numerico(self,cadena:str):
        regex=re.compile('(Con\sPaso\s[0-9]+)')
        if re.fullmatch(regex,cadena):
            return True
        else:
            return False
        
    def condicion(self,cadena:str):
        """Esta función valida que la asignación sea valida
        """
        regex=re.compile('[a-zA-Z0-9_]+=([a-zA-Z0-9_]+)')
        if re.fullmatch(regex,cadena):
            return True
        else:
            return False
    
    def condicion2(self,cadena:str):
        """Esta función valida que la condición sea valida
        """
        regex=re.compile('[a-zA-Z0-9_]+(==|<|>|<=|>=)([a-zA-Z0-9_]+)')
        if re.fullmatch(regex,cadena):
            return True
        else:
            return False
    
    def instrucciones(self,cadena:str):
        regex=re.compile('((([a-zA-Z0-9_(\s)+]+)(=|<|>|<=|>=)([a-zA-Z0-9_(\s+)(;*)]+)))|([a-zA-Z0-9_(\s)+])')
        if re.fullmatch(regex,cadena):
            return True
        else:
            return False
    
    def escritura(self,cadena:str):
        regex = re.compile('(Escribir\s+([a-zA-Z0-9_]+,)*[a-zA-Z0-9_]+;)|(Escribir)')
        if re.fullmatch(regex,cadena):
            return True
        else:   
            return False
    
    def si(self):
        instruc=''
        estructura_actual = self.codigo
        cadena_actual = self.obtener_siguiente_cadena(estructura_actual)
        if cadena_actual and cadena_actual == "Si":
            cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
            if cadena_actual and self.instrucciones(cadena_actual):
                cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                if cadena_actual and cadena_actual == "Entonces":
                    cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                    while cadena_actual: 
                        instruc=instruc+cadena_actual      
                        if cadena_actual and (self.escritura(cadena_actual) or self.instrucciones(cadena_actual)):
                            cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                        else:
                            print('Error, se esperaba una instrucción valida')
                            self.lanzar_excepcion("Error , se esperaba una instrucción valida")
                        if cadena_actual=='FinSi':
                            break           
                    print(instruc)
                    if cadena_actual !='FinSi':
                        print('Error, se esperaba la palabra "FinSi"')
                        self.lanzar_excepcion("Error , se esperaba la palabra 'FinSi'")
                    else:
                        print('Finalizado')
                        self.reiniciar()
                else:
                    print("Error , se esperaba obtener la palabra 'Entonces'")
                    self.lanzar_excepcion("Error , se esperaba obtener la palabra 'Entonces'")
            else:
                print('Error, se esperaba una condicion valida')
                self.lanzar_excepcion("Error , se esperaba obtener una condicion valida")
        else:
            print('No hice nada if')
            self.reiniciar()
               
    def ciclofor(self):
        estructura_actual = self.codigo
        cadena_actual = self.obtener_siguiente_cadena(estructura_actual)
        if cadena_actual and cadena_actual=='Para':
            cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
            if cadena_actual and self.condicion(cadena_actual):
                cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                if cadena_actual and cadena_actual=='Hasta':
                    cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                    if cadena_actual and self.valor_numerico(cadena_actual):
                        cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                        if cadena_actual and cadena_actual=='(':
                            cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                            if cadena_actual and cadena_actual=='Con':
                                cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                                if cadena_actual and cadena_actual=='Paso':
                                    cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                                    if cadena_actual and self.valor_numerico(cadena_actual):
                                        cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                                        if cadena_actual and cadena_actual==')':
                                            cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                                            if cadena_actual and cadena_actual=='Hacer':
                                                cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                                                while cadena_actual:
                                                    if cadena_actual and (self.escritura(cadena_actual) or self.instrucciones(cadena_actual)):
                                                        cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                                                    else:
                                                        print('Error, se esperaba una instrucción valida')
                                                        self.lanzar_excepcion("Error , se esperaba una instrucción valida")
                                                    if cadena_actual=='FinPara':
                                                        break           
                                                if cadena_actual !='FinPara':
                                                    print('Error, se esperaba la palabra "FinPara"')
                                                    self.lanzar_excepcion("Error , se esperaba la palabra 'FinPara'")
                                                else:
                                                    print('Finalizado')
                                                    self.reiniciar()
                                                    
                                            else:
                                                print('Error, se esperaba la palabra "Entonces"')
                                                self.lanzar_excepcion("Error , se esperaba la palabra 'Entonces'")
                                        else:
                                            print('Error, se esperaba un parentesis de cierre')
                                            self.lanzar_excepcion("Error , se esperaba un parentesis de cierre")
                                    else:
                                        print('Error, se esperaba un valor numerico')
                                        self.lanzar_excepcion("Error , se esperaba un valor numerico")
                                else:
                                    print('Error, se esperaba la palabra "Paso"')
                                    self.lanzar_excepcion("Error , se esperaba la palabra 'Paso'")
                            else:
                                print('Error, se esperaba la palabra "Con"')
                                self.lanzar_excepcion("Error , se esperaba la palabra 'Con'")
                        else:
                            print('Error, se esperaba un parentesis de apertura')
                            self.lanzar_excepcion('Error, se esperaba un parentesis de apertura')
                    else:
                        print('Error, se esperaba un final del loop valido')
                        self.lanzar_excepcion("Error, se esperaba un final del loop valido")
                else:
                    print('Error, se esperaba la palabra "Hasta"')
                    self.lanzar_excepcion("Error , se esperaba la palabra 'Hasta'")
            else:
                print('Error, se esperaba una condicion valida')
                self.lanzar_excepcion("Error , se esperaba obtener una condicion valida")
        else:
            print('No hice nada for')
            self.reiniciar()
    
    def ciclowhile(self):
        estructura_actual = self.codigo
        cadena_actual = self.obtener_siguiente_cadena(estructura_actual)
        if cadena_actual and cadena_actual=='Mientras':
            cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
            if cadena_actual and self.condicion2(cadena_actual):
                cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                if cadena_actual and cadena_actual=='Hacer':
                    cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                    while cadena_actual:
                        if cadena_actual and (self.escritura(cadena_actual) or self.instrucciones(cadena_actual)):
                            cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                        else:
                            print('Error, se esperaba una instrucción valida')
                            self.lanzar_excepcion("Error , se esperaba una instrucción valida")
                        if cadena_actual=='FinMientras':
                            break           
                    if cadena_actual !='FinMientras':
                        print('Error, se esperaba la palabra "FinMientras"')
                        self.lanzar_excepcion("Error , se esperaba la palabra 'FinMientras'")
                    else:
                        print('Finalizado')
                        self.reiniciar()
                else:
                    print('Error, se esperaba la palabra "Hacer"')
                    self.lanzar_excepcion("Error , se esperaba la palabra 'Hacer'")
            else:
                print('Error, se esperaba una condicion valida')
                self.lanzar_excepcion("Error , se esperaba obtener una condicion valida")
        else:
            print('No hice nada while')
            self.reiniciar()

    def dowhile(self):
        instruc=''
        estructura_actual = self.codigo
        cadena_actual = self.obtener_siguiente_cadena(estructura_actual)
        if cadena_actual and cadena_actual=='Repetir':
            cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
            while cadena_actual: 
                instruc=instruc+cadena_actual      
                if cadena_actual and (self.escritura(cadena_actual) or self.instrucciones(cadena_actual)):
                    cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                else:
                    print('Error, se esperaba una instrucción valida')
                    self.lanzar_excepcion("Error , se esperaba una instrucción valida")
                if cadena_actual !='Hasta':
                    print('Error, se esperaba la palabra "Hasta Que"')
                    self.lanzar_excepcion("Error , se esperaba la palabra 'Hasta Que'")
                else:
                    break
            cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
            if cadena_actual and cadena_actual=='Que':
                cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                if cadena_actual and self.condicion2(cadena_actual):
                    print('Finalizado')
                    self.reiniciar()  
                else:
                    print('Error, se esperaba una condicion valida')
                    self.lanzar_excepcion("Error , se esperaba obtener una condicion valida")
            else:
                print('Error, se esperaba la palabra "Hasta Que"')
                self.lanzar_excepcion("Error , se esperaba la palabra 'Hasta Que'")
        else:
            print('No hice nada dowhile')
            self.reiniciar()

    def asignacion(self):
        estructura_actual = self.codigo
        cadena_actual = self.obtener_siguiente_cadena(estructura_actual)
        print(cadena_actual)
        if cadena_actual and self.alfanumerico(cadena_actual) and cadena_actual!='Si' and cadena_actual!='Mientras' and cadena_actual!='Para' and cadena_actual!='Repetir' and cadena_actual!='Escribir':
            cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
            if cadena_actual and cadena_actual=='=':
                cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                if cadena_actual and self.alfanumerico(cadena_actual):
                    cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
            else:
                print('Error, se esperaba un signo igual')
                self.lanzar_excepcion("Error , se esperaba un signo igual")
            if cadena_actual and cadena_actual==';':
                print('Finalizado')
            else:
                print('Error, se esperaba un punto y coma')
                self.lanzar_excepcion("Error , se esperaba un punto y coma")
        else:
            print('No hice nada asignacion')
            self.reiniciar()
    
    def escritura2(self):
        estructura_actual = self.codigo
        cadena_actual = self.obtener_siguiente_cadena(estructura_actual)
        if cadena_actual and cadena_actual=='Escribir':
            cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
            if cadena_actual and self.alfanumerico(cadena_actual) and cadena_actual!='Si' and cadena_actual!='Mientras' and cadena_actual!='Para' and cadena_actual!='Repetir':
                cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
            else:
                print('Error, se esperaba una variable')
                self.lanzar_excepcion("Error , se esperaba una variable")
        else:
            print('No hice nada escritura')
            self.reiniciar()
          
    def lanzar_excepcion(self,mensaje:str):
        """Esta función lanza una excepción al encontrar un error en la cadena de entrada

        :param mensaje: El mensaje que quiere ser mostrado al usuario
        :type mensaje: str
        :raises self.error: Esta excepción es lanzada para indicar que se encontró un error
        """
        self.error_actual=mensaje
        raise error(self.error_actual)
    
    def analisis(self):
        """Esta función analiza la cadena de entrada y verifica que sea valida
        """
        self.si()
        self.ciclofor()
        self.ciclowhile()
        self.dowhile()
        self.asignacion()
        self.escritura2()
        print('------------------------')
        print('Finalizado')

## GUI

In [ ]:
import tkinter as tk
from tkinter import messagebox
from ttkbootstrap import Style
import ttkbootstrap as tb

class GUIApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Analizador léxico-sintáctico")
        self.geometry("700x500")
        self.pseudocodigo=[]

        self.style = Style(theme='vapor')

        self.titulo=tb.Label(text='Analizador léxico-sintáctico',font=('Verdana',18),bootstyle='light')
        self.titulo.pack(side=tb.TOP,pady=15)

        self.contenedor = tb.Labelframe(text='Pseudocódigo', padding=35)
        self.contenedor.pack(side=tb.TOP,padx=(20,20),pady=(0,20))

        self.text_input = tb.Text(master=self.contenedor, width=70,height=10)
        self.text_input.pack(pady=10)


        self.buttonbar = tb.Frame()
        self.buttonbar.pack()

        self.b1 = tb.Button(master=self.buttonbar,width=20, text='Revisar', bootstyle='light',command=self.revisar)
        self.b1.pack(side=tb.LEFT,padx=10)

        self.b2=tb.Button(master=self.buttonbar,width=20,text='Borrar',bootstyle='light',command=self.borrar)
        self.b2.pack(side=tb.LEFT,padx=10)

        self.barra=tb.Progressbar(bootstyle="success-striped",maximum=101,value=0)
        self.barra.pack(pady=10)

        self.status_label=tb.Label(text='Status : ',font=('Verdana',18),bootstyle='light')
        self.status_label.pack(pady=10)


    def revisar(self):
        self.analizadorlexico=AnalizadorLexico()
        self.barra.step(100)  # Set the step value to the maximum value (100)
        self.pseudocodigo=self.analizadorlexico.analisis(self.text_input.get("1.0","end-1c"))
        self.pseudocodigo2=[self.text_input.get("1.0","end-1c")]
        self.pseudocodigo.append(self.pseudocodigo2)
        print(self.pseudocodigo)
        print(type(self.pseudocodigo))
        if len(self.pseudocodigo)==0:
            messagebox.showerror(message='No se ha ingresado ningún pseudocódigo', title="Error")
            self.status_label.config(text="Status: Error")
        else:
            for codigo in self.pseudocodigo:
                if len(codigo)==0:
                    pass
                else:
                    self.analizadorsintactico=AnalizadorSintactico(codigo)
                    messagebox.showinfo(message='Se ha revisado su pseudocódigo¡¡', title="Completado")
                    self.status_label.config(text="Status: Tu código ha sido revisado")
                    try :
                        self.analizadorsintactico.analisis()
                    except error as e:
                        mensaje=e.__str__()
                        messagebox.showerror(message=mensaje, title="Error")
                        break
                    messagebox.showinfo(message='Tu pseudocodigo esta correcto 👍', title="Completado")

    def borrar(self):
        self.status_label.config(text="Status: Borrarado")
        self.text_input.delete("1.0","end-1c")
        self.barra.stop()
        self.barra["value"] = 0
        self.pseudocodigo=[]

    def run(self):
        self.mainloop()

if __name__ == '__main__':
    app = GUIApp()
    app.run()